In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import inspect

In [36]:
for element in inspect.getmembers(torch.nn.modules.Module):
    print("{}\n".format(element))

('__call__', <function Module.__call__ at 0x7f38d2c85400>)

('__class__', <class 'type'>)

('__delattr__', <function Module.__delattr__ at 0x7f38d2c85620>)

('__dict__', mappingproxy({'__module__': 'torch.nn.modules.module', '__doc__': 'Base class for all neural network modules.\n\n    Your models should also subclass this class.\n\n    Modules can also contain other Modules, allowing to nest them in\n    a tree structure. You can assign the submodules as regular attributes::\n\n        import torch.nn as nn\n        import torch.nn.functional as F\n\n        class Model(nn.Module):\n            def __init__(self):\n                super(Model, self).__init__()\n                self.conv1 = nn.Conv2d(1, 20, 5)\n                self.conv2 = nn.Conv2d(20, 20, 5)\n\n            def forward(self, x):\n               x = F.relu(self.conv1(x))\n               return F.relu(self.conv2(x))\n\n    Submodules assigned in this way will be registered, and will have their\n    parameters converted 

# Pytorch `Module` class

![torch nn modules module](https://user-images.githubusercontent.com/13328380/51440701-33105700-1d0d-11e9-9611-6bd5c517433c.png)

# NOTE
1. `:meth:`는 `method`를 의미함

Init signature: torch.nn.modules.Module()
Docstring:     
## Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::
```python
    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super(Model, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
           x = F.relu(self.conv1(x))
           return F.relu(self.conv2(x))
```

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.
    
File:           ~/.local/lib/python3.6/site-packages/torch/nn/modules/module.py  
Type:           type  
Subclasses:     Linear, Bilinear, _ConvNd, Threshold, RReLU, Hardtanh, Sigmoid, Tanh, ELU, CELU, ...  

In [ ]:
# Field
# what is mean?
dump_patches = False

In [ ]:
# Field
_version = 1

This allows better BC support for :meth:`load_state_dict`. In
    :meth:`state_dict`, the version number will be saved as in the attribute
    `_metadata` of the returned state dict, and thus pickled. `_metadata` is a
    dictionary with keys that follow the naming convention of state dict. See
    ``_load_from_state_dict`` on how to use this information in loading.

    If new parameters/buffers are added/removed from a module, this number shall
    be bumped, and the module's `_load_from_state_dict` method can compare the
    version number and do appropriate changes if the state dict is from before
    the change.

`load_state_dict` method에 대한 BC 지원이 향상된다. `state_dict` method에서, 버전 번호는 return된 state dict의 `_metadata` 내부의 속성으로써 저장될 것이고, pickled된다.

`_metadata`는 state dict의 naming convention을 따르는 key가 있는 dictionary다. 해당 정보를 어떻게 사용하는지 보고싶으면`_load_from_state_dict`을 봐라.

만약 모듈에서 새로운 파라미터/버퍼가 추가되거나 제거된다면 해당 숫자는 충돌을 일으켜야한다.
모듈의 `_load_from_state_dict` method는 버전 번호를 비교할 수 있고, 만약에 state dict이 이전 버전에서 온 경우, 이를 적절하게 변경할 수 있다.

In [24]:
def __init__(self):
        self._backend = thnn_backend
        self._parameters = OrderedDict()
        self._buffers = OrderedDict()
        self._backward_hooks = OrderedDict()
        self._forward_hooks = OrderedDict()
        self._forward_pre_hooks = OrderedDict()
        self._state_dict_hooks = OrderedDict()
        self._load_state_dict_pre_hooks = OrderedDict()
        self._modules = OrderedDict()
        self.training = True

In [25]:
 def forward(self, *input):
        r"""Defines the computation performed at every call.

        Should be overridden by all subclasses.

        .. note::
            Although the recipe for forward pass needs to be defined within
            this function, one should call the :class:`Module` instance afterwards
            instead of this since the former takes care of running the
            registered hooks while the latter silently ignores them.
        """
        raise NotImplementedError

Signature: torch.nn.modules.Module.forward(self, *input)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.  
    
File:      ~/.local/lib/python3.6/site-packages/torch/nn/modules/module.py  
Type:      function


모든 subclass들에 의해서 override되어야 한다.

_**무슨 말일까?....**_
>forwad pass를 위한 recipe는 `forward` method와 같이 정의되어야하지만  latter는 등록된 hook을 조용히 무시하는 반면에 former는 등록된 hook을 실행하기 때문에 Module Class instance는 나중에 호출할 필요가 있다.


In [ ]:
  def register_buffer(self, name, tensor):
        r"""Adds a persistent buffer to the module.

        This is typically used to register a buffer that should not to be
        considered a model parameter. For example, BatchNorm's ``running_mean``
        is not a parameter, but is part of the persistent state.

        Buffers can be accessed as attributes using given names.

        Args:
            name (string): name of the buffer. The buffer can be accessed
                from this module using the given name
            tensor (Tensor): buffer to be registered.

        Example::

            >>> self.register_buffer('running_mean', torch.zeros(num_features))

        """
        if not isinstance(name, torch._six.string_classes):
            raise TypeError("buffer name should be a string. "
                            "Got {}".format(torch.typename(name)))
        elif '.' in name:
            raise KeyError("buffer name can't contain \".\"")
        elif name == '':
            raise KeyError("buffer name can't be empty string \"\"")
        elif hasattr(self, name) and name not in self._buffers:
            raise KeyError("attribute '{}' already exists".format(name))
        elif tensor is not None and not isinstance(tensor, torch.Tensor):
            raise TypeError("cannot assign '{}' object to buffer '{}' "
                            "(torch Tensor or None required)"
                            .format(torch.typename(tensor), name))
        else:
            self._buffers[name] = tensor

Signature: torch.nn.modules.Module.register_buffer(self, name, tensor)
Docstring:
Adds a persistent buffer to the module.

This is typically used to register a buffer that should not to be
considered a model parameter. For example, BatchNorm's ``running_mean``
is not a parameter, but is part of the persistent state.

Buffers can be accessed as attributes using given names.

Args:
    name (string): name of the buffer. The buffer can be accessed
        from this module using the given name
    tensor (Tensor): buffer to be registered.

Example::

    >>> self.register_buffer('running_mean', torch.zeros(num_features))
    
File:      ~/.local/lib/python3.6/site-packages/torch/nn/modules/module.py  
Type:      function  

해당 함수는 특별하게 model parameter로 고려되면 안되는 파라미터(buffer)를 등록하는데 사용된다.
> 예를들어서 BatchNorm의 `running_mean`은 prameter가 아니지만 지속성있는 상태(persistent state)의 일부다.

Buffer들은 주어진 이름을 사용해서 attribute로써 접근될 수 있다.

Args: name(string): buffer의 이름이다. Buffer는 지정할 이름과, Tensor를 이용해서 등록할 수 있다.
Example ::

```python
self.register_buffer('running_mean', torch.zeros(num_features))
```

In [37]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 20, 5, 1)
        self.conv2 = torch.nn.Conv2d(20, 50, 5, 1)
        self.fc1 = torch.nn.Linear(4*4*50, 500)
        self.fc2 = torch.nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [39]:
model = Model()
print(model._buffers)
model.register_buffer('running_mean', torch.zeros(10))
print(model._buffers)

OrderedDict()
OrderedDict([('running_mean', tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))])
